In [2]:
import numpy as np
import pandas as pd
import os
import matplotlib.pyplot as plt
import csv
import datetime
import urllib
import zipfile
import scipy.stats as stat
%pylab inline

Populating the interactive namespace from numpy and matplotlib


In [3]:
#borrowed this code, download and unzip
def getunzipped(theurl, thedir):
  name = os.path.join(thedir, 'temp.zip')
  try:
    name, hdrs = urllib.urlretrieve(theurl, name)
  except IOError, e:
    print "Can't retrieve %r to %r: %s" % (theurl, thedir, e)
    return
  try:
    z = zipfile.ZipFile(name)
  except zipfile.error, e:
    print "Bad zipfile (from %r): %s" % (theurl, e)
    return
  for n in z.namelist():
    dest = os.path.join(thedir, n)
    destdir = os.path.dirname(dest)
    if not os.path.isdir(destdir):
      os.makedirs(destdir)
    data = z.read(n)
    f = open(dest, 'w')
    f.write(data)
    f.close()
  z.close()
  os.unlink(name)
    
    

In [4]:
#check to see if file already exists in directory
if os.path.isfile('2014-02 - Citi Bike trip data.csv') is False:
    getunzipped('https://s3.amazonaws.com/tripdata/201402-citibike-tripdata.zip',os.getcwd())

if os.path.isfile('2014-08 - Citi Bike trip data.csv') is False:
    getunzipped('https://s3.amazonaws.com/tripdata/201408-citibike-tripdata.zip',os.getcwd())


In [5]:
#read csv's

df0 = pd.read_csv('2014-02 - Citi Bike trip data.csv')
df1 = pd.read_csv('2014-08 - Citi Bike trip data.csv')

In [6]:
#we are doing a two sample test of proportions, so:

def ztest_twosides(teststat, nullvalue,  se, alpha):
    Ztest = (teststat-nullvalue)/se
    pvalue = stat.norm.pdf(Ztest)
    if Ztest > 0.: #this considers both sides of 0, e.g. when Z is 1SD, the value ~ .8413
        pvalue = 1. -pvalue
    pvalue *= 2.
    return program_output(pvalue, alpha)

def two_proportions(x0, n0, x1, n1, alpha=0.05):
    p0 = float(x0)/n0
    p1 = float(x1)/n1
    p = float(x0 +x1)/(n0+n1)
    samplestat =  p0 - p1 #sample statistic
    se = sqrt(p*(1.0-p) * (1.0/n0 + 1.0/n1))
    print 'p_0 = %f and p_1= %f' %(p0,p1)
    return ztest_twosides(samplestat,0,  se,  alpha)

def program_output(pvalue,alpha):
    print 'The Null hypothesis is {}'.format( 'rejected.' if pvalue<alpha  else 'not rejected.') 
    print 'The p-value is %f.' %(pvalue)


In [7]:
#fraction of 'tourists' is the desired proportion, and we'd liek to see what happend with the 'treatment', i.e. summer.
x0 = df0['usertype'].value_counts()[1]  #number of customers
x1 = df1['usertype'].value_counts()[1]
n0 = len(df0.index)
n1 = len(df1.index)
s0 = n0-x0 #no. of subscribers
s1 = n1-x1 

In [8]:
two_proportions(x0, n0, x1, n1, 0.01) #should return p-value, z, and critical values


p_0 = 0.029888 and p_1= 0.141023
The Null hypothesis is rejected.
The p-value is 0.000000.


We have completed the assignment but are now interested in a ChiSquared test, just because.

In [10]:
s0,x0

(218019, 6717)

In [12]:
import scipy.stats
house = [ [ x0, x1 ], [ s0, s1 ] ]
chi2, p, ddof, expected = scipy.stats.chi2_contingency( house )
msg = "Test Statistic: {}\np-value: {}\nDegrees of Freedom: {}\n"
print( msg.format( chi2, p, ddof ) )
print( expected )

Test Statistic: 21311.9168843
p-value: 0.0
Degrees of Freedom: 1

[[  26969.07654358  115621.92345642]
 [ 197766.92345642  847867.07654358]]


In [11]:
#github can't handle it
os.remove('2014-02 - Citi Bike trip data.csv')
os.remove('2014-08 - Citi Bike trip data.csv')

In [2]:
#maybe if we wish to do additional testing especially ChiSq, we'd look at day of week and proportion of 'Customers'?
def weekday_maker(df,column_name):
    #create error controls here
    df[column_name] = pd.to_datetime(df[column_name])
    df['weekday'] = df[column_name].dt.dayofweek #create day of week
    df['date']=df[column_name].apply(lambda x:x.date()) #create date for grouping
    days = {0:'Mon',1:'Tues',2:'Weds',3:'Thurs',4:'Fri',5:'Sat',6:'Sun'}
    df['weekday'] = df['weekday'].apply(lambda x: days[x])
    


In [3]:
weekday_maker(df1,'starttime')
weekday_maker(df0,'starttime')

NameError: name 'df1' is not defined

In [20]:
df0.groupby(['date']).mean.apply(len)

AttributeError: 'function' object has no attribute 'apply'

In [24]:
read_csv